In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [14]:
"""Trains a ResNet on the CIFAR10 dataset.

ResNet v1
[a] Deep Residual Learning for Image Recognition
https://arxiv.org/pdf/1512.03385.pdf

ResNet v2
[b] Identity Mappings in Deep Residual Networks
https://arxiv.org/pdf/1603.05027.pdf
"""

from __future__ import print_function
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
from keras.callbacks import LambdaCallback
from keras.callbacks import CSVLogger
from keras.callbacks import Callback
from keras.preprocessing.image import ImageDataGenerator
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
import numpy as np
import pickle
import math
from collections import defaultdict
import os

dirpath = os.getcwd()

seed = 22
resample = True
save_dir = '/home/ubuntu/Projects/hybrid-ensemble/model/run_200'
datafile = '/home/ubuntu/Projects/hybrid-ensemble/data/cifar10_balance/DS3'
top_k = 1

# Set random seed
if seed is not None:
    import tensorflow as tf
    tf.set_random_seed(seed)
    np.random.seed(seed)

# Training parameters
batch_size = 128  # orig paper trained all networks with batch_size=128
epochs = 5 # orig paper epochs = 200
data_augmentation = True
num_classes = 10
initial_lr = 1e-3
snapshot_window_size = int(math.ceil(epochs/top_k))

# Subtracting pixel mean improves accuracy
subtract_pixel_mean = True

# Model parameter
# ----------------------------------------------------------------------------
#           |      | 200-epoch | Orig Paper| 200-epoch | Orig Paper| sec/epoch
# Model     |  n   | ResNet v1 | ResNet v1 | ResNet v2 | ResNet v2 | GTX1080Ti
#           |v1(v2)| %Accuracy | %Accuracy | %Accuracy | %Accuracy | v1 (v2)
# ----------------------------------------------------------------------------
# ResNet20  | 3 (2)| 92.16     | 91.25     | -----     | -----     | 35 (---)
# ResNet32  | 5(NA)| 92.46     | 92.49     | NA        | NA        | 50 ( NA)
# ResNet44  | 7(NA)| 92.50     | 92.83     | NA        | NA        | 70 ( NA)
# ResNet56  | 9 (6)| 92.71     | 93.03     | 93.01     | NA        | 90 (100)
# ResNet110 |18(12)| 92.65     | 93.39+-.16| 93.15     | 93.63     | 165(180)
# ResNet164 |27(18)| -----     | 94.07     | -----     | 94.54     | ---(---)
# ResNet1001| (111)| -----     | 92.39     | -----     | 95.08+-.14| ---(---)
# ---------------------------------------------------------------------------
n = 3

# Model version
# Orig paper: version = 1 (ResNet v1), Improved ResNet: version = 2 (ResNet v2)
version = 1

# Computed depth from supplied model parameter n
if version == 1:
    depth = n * 6 + 2
elif version == 2:
    depth = n * 9 + 2

# Model name, depth and version
model_type = 'ResNet%dv%d' % (depth, version)

# Load the CIFAR10 data.
print('Loading data...')
with open(datafile, 'rb') as f:
    (x_train, y_train), (x_valid, y_valid), (x_test, y_test) = pickle.load(f)

# Resample the training data set from training+validating data set with the same class distribution with the loaded ones
if resample:
    print('Resampling training and validating data sets...')
    x_tv = np.concatenate((x_train, x_valid), axis=0)
    y_tv = np.concatenate((y_train, y_valid), axis=0)
    index_dict = defaultdict(list)
    for i in range(len(y_tv)):
        index_dict[y_tv[i][0]].append(i)
    valid_index_dict = defaultdict(list)
    for i in range(len(y_valid)):
        valid_index_dict[y_valid[i][0]].append(i)
    valid_index = []
    for c in valid_index_dict.keys():
        valid_index.extend(np.random.choice(index_dict[c], size=len(valid_index_dict[c]), replace=False))
    train_index = np.setdiff1d(range(len(y_tv)), valid_index)

    x_train, y_train = x_tv[train_index], y_tv[train_index]
    x_valid, y_valid = x_tv[valid_index], y_tv[valid_index]
    
# Input image dimensions.
input_shape = x_train.shape[1:]

# Normalize data.
x_train = x_train.astype('float32') / 255
x_test = x_test.astype('float32') / 255
x_valid = x_valid.astype('float32') / 255

# If subtract pixel mean is enabled
if subtract_pixel_mean:
    x_train_mean = np.mean(x_train, axis=0)
    x_train -= x_train_mean
    x_test -= x_train_mean
    x_valid -= x_train_mean

print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')
print('y_train shape:', y_train.shape)
print(x_valid.shape[0], 'valid samples')

# Convert class vectors to binary class matrices.
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)
y_valid = keras.utils.to_categorical(y_valid, num_classes)

def next_run_dir(path):
    """
    Naive (slow) version of next_path
    """
    i = 1
    while os.path.exists('{}_{}'.format(path, i)):
        i += 1
    return '{}_{}'.format(path, i)

def cyclic_cosine_anneal_schedule(initial_lr=1e-3, update_window_size=40):
    '''
    Wrapper function to create a LearningRateScheduler with cosine annealing schedule.
    '''
    def lr_schedule(epoch):
        """Learning Rate Schedule

        Learning rate is scheduled to be updated per epoch with a cosine function per epoch. 
        Learning rate is raised to initial_lr every snapshot_window_size.

        # Arguments
            epoch (int): The number of epochs

        # Returns
            lr (float32): learning rate
        """
        lr = initial_lr / 2 * (math.cos(math.pi * ((epoch % update_window_size) / update_window_size)) + 1)
        print('Learning rate: ', lr)
        return lr
    
    return LearningRateScheduler(lr_schedule)

class MyCallback(Callback):
    def on_batch_begin(self, batch, logs=None):
        iteration = self.model.optimizer.iterations   
        update_window_size = self.params.update_window_size
        lr = initial_lr / 2 * (math.cos(math.pi * ((K.eval(iteration) % update_window_size) / update_window_size)) + 1)
        K.set_value(self.model.optimizer.lr, lr)
        print('\n batch {}, lr {}, iterations {}'.format(batch, K.eval(lr), K.eval(iteration)))
        
def cyclic_cosine_anneal_schedule_itr(batch_logs, initial_lr=1e-3, update_window_size=15600):
    '''
    Wrapper function to create a LearningRateScheduler with cosine annealing schedule per iteration.
    '''
    def lr_schedule(batch, logs):
        """Learning Rate Schedule

        Learning rate is scheduled to be updated per epoch with a cosine function per iteration. 
        Learning rate is raised to initial_lr every snapshot_window_size.

        # Arguments
            epoch (int): The number of epochs

        # Returns
            lr (float32): learning rate
        """
        iteration = model.optimizer.iterations
        print('\n Optimizer iteration {}, batch {}'.format(K.eval(iteration), batch))
        lr = initial_lr / 2 * (math.cos(math.pi * ((K.eval(iteration) % update_window_size) / update_window_size)) + 1)
        K.set_value(model.optimizer.lr, lr)
        print('\n Learning rate {}, Model learning rate {}'.format(lr, K.eval(model.optimizer.lr)))
    
    def batch_log(batch, logs):
        batch_logs['iteration'].append(K.eval(model.optimizer.iterations))
        batch_logs['lr'].append(K.eval(model.optimizer.lr))
        batch_logs['loss'].append(logs['loss'])
        batch_logs['acc'].append(logs['acc'])
        
    
    return LambdaCallback(on_batch_begin=lr_schedule, on_batch_end=batch_log)

def resnet_layer(inputs,
                 num_filters=16,
                 kernel_size=3,
                 strides=1,
                 activation='relu',
                 batch_normalization=True,
                 conv_first=True):
    """2D Convolution-Batch Normalization-Activation stack builder

    # Arguments
        inputs (tensor): input tensor from input image or previous layer
        num_filters (int): Conv2D number of filters
        kernel_size (int): Conv2D square kernel dimensions
        strides (int): Conv2D square stride dimensions
        activation (string): activation name
        batch_normalization (bool): whether to include batch normalization
        conv_first (bool): conv-bn-activation (True) or
            activation-bn-conv (False)

    # Returns
        x (tensor): tensor as input to the next layer
    """
    conv = Conv2D(num_filters,
                  kernel_size=kernel_size,
                  strides=strides,
                  padding='same',
                  kernel_initializer='he_normal',
                  kernel_regularizer=l2(1e-4))

    x = inputs
    if conv_first:
        x = conv(x)
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
    else:
        if batch_normalization:
            x = BatchNormalization()(x)
        if activation is not None:
            x = Activation(activation)(x)
        x = conv(x)
    return x


def resnet_v1(input_shape, depth, num_classes=10):
    """ResNet Version 1 Model builder [a]

    Stacks of 2 x (3 x 3) Conv2D-BN-ReLU
    Last ReLU is after the shortcut connection.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filters is
    doubled. Within each stage, the layers have the same number filters and the
    same number of filters.
    Features maps sizes:
    stage 0: 32x32, 16
    stage 1: 16x16, 32
    stage 2:  8x8,  64
    The Number of parameters is approx the same as Table 6 of [a]:
    ResNet20 0.27M
    ResNet32 0.46M
    ResNet44 0.66M
    ResNet56 0.85M
    ResNet110 1.7M

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 6 != 0:
        raise ValueError('depth should be 6n+2 (eg 20, 32, 44 in [a])')
    # Start model definition.
    num_filters = 16
    num_res_blocks = int((depth - 2) / 6)

    inputs = Input(shape=input_shape)
    x = resnet_layer(inputs=inputs)
    # Instantiate the stack of residual units
    for stack in range(3):
        for res_block in range(num_res_blocks):
            strides = 1
            if stack > 0 and res_block == 0:  # first layer but not first stack
                strides = 2  # downsample
            y = resnet_layer(inputs=x,
                             num_filters=num_filters,
                             strides=strides)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters,
                             activation=None)
            if stack > 0 and res_block == 0:  # first layer but not first stack
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])
            x = Activation('relu')(x)
        num_filters *= 2

    # Add classifier on top.
    # v1 does not use BN after last shortcut connection-ReLU
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model


def resnet_v2(input_shape, depth, num_classes=10):
    """ResNet Version 2 Model builder [b]

    Stacks of (1 x 1)-(3 x 3)-(1 x 1) BN-ReLU-Conv2D or also known as
    bottleneck layer
    First shortcut connection per layer is 1 x 1 Conv2D.
    Second and onwards shortcut connection is identity.
    At the beginning of each stage, the feature map size is halved (downsampled)
    by a convolutional layer with strides=2, while the number of filter maps is
    doubled. Within each stage, the layers have the same number filters and the
    same filter map sizes.
    Features maps sizes:
    conv1  : 32x32,  16
    stage 0: 32x32,  64
    stage 1: 16x16, 128
    stage 2:  8x8,  256

    # Arguments
        input_shape (tensor): shape of input image tensor
        depth (int): number of core convolutional layers
        num_classes (int): number of classes (CIFAR10 has 10)

    # Returns
        model (Model): Keras model instance
    """
    if (depth - 2) % 9 != 0:
        raise ValueError('depth should be 9n+2 (eg 56 or 110 in [b])')
    # Start model definition.
    num_filters_in = 16
    num_res_blocks = int((depth - 2) / 9)

    inputs = Input(shape=input_shape)
    # v2 performs Conv2D with BN-ReLU on input before splitting into 2 paths
    x = resnet_layer(inputs=inputs,
                     num_filters=num_filters_in,
                     conv_first=True)

    # Instantiate the stack of residual units
    for stage in range(3):
        for res_block in range(num_res_blocks):
            activation = 'relu'
            batch_normalization = True
            strides = 1
            if stage == 0:
                num_filters_out = num_filters_in * 4
                if res_block == 0:  # first layer and first stage
                    activation = None
                    batch_normalization = False
            else:
                num_filters_out = num_filters_in * 2
                if res_block == 0:  # first layer but not first stage
                    strides = 2    # downsample

            # bottleneck residual unit
            y = resnet_layer(inputs=x,
                             num_filters=num_filters_in,
                             kernel_size=1,
                             strides=strides,
                             activation=activation,
                             batch_normalization=batch_normalization,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_in,
                             conv_first=False)
            y = resnet_layer(inputs=y,
                             num_filters=num_filters_out,
                             kernel_size=1,
                             conv_first=False)
            if res_block == 0:
                # linear projection residual shortcut connection to match
                # changed dims
                x = resnet_layer(inputs=x,
                                 num_filters=num_filters_out,
                                 kernel_size=1,
                                 strides=strides,
                                 activation=None,
                                 batch_normalization=False)
            x = keras.layers.add([x, y])

        num_filters_in = num_filters_out

    # Add classifier on top.
    # v2 has BN-ReLU before Pooling
    x = BatchNormalization()(x)
    x = Activation('relu')(x)
    x = AveragePooling2D(pool_size=8)(x)
    y = Flatten()(x)
    outputs = Dense(num_classes,
                    activation='softmax',
                    kernel_initializer='he_normal')(y)

    # Instantiate model.
    model = Model(inputs=inputs, outputs=outputs)
    return model

print('Building model...')
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=initial_lr),
              metrics=['accuracy'])
# model.summary()
print(model_type)

# Prepare model model saving directory.
if not save_dir:
    save_dir = next_run_dir('{}/../model/run'.format(dirpath))
os.makedirs(save_dir)
model_name = 'cifar10_%s_model-{epoch:04d}.h5' % model_type
# model_name = 'cifar10_%s_model-{epoch:04d}-{val_acc:.5f}.h5' % model_type
# model_name = 'cifar10_{}_model.h5'.format(model_type)
filepath = os.path.join(save_dir, model_name)

print('Preparing callbacks...')
# Prepare callbacks for model saving and for learning rate adjustment.
checkpoint = ModelCheckpoint(filepath=filepath,
                             monitor='val_acc',
                             verbose=1,
                             save_best_only=False,
                             mode='max')
# Learning rate updater
batch_num = int(x_train.shape[0]/batch_size)
update_window_size = int(math.ceil(epochs*batch_num/top_k))
batch_logs = {'iteration':[], 'lr':[], 'loss':[], 'acc':[]}
lr_scheduler = cyclic_cosine_anneal_schedule_itr(initial_lr=initial_lr, 
                                                 update_window_size=update_window_size, batch_logs=batch_logs)

# Training log writer
csvlog = CSVLogger('callback_training_log.csv', separator=',', append=False)


callbacks = [checkpoint, lr_scheduler, csvlog]

# callbacks = [MyCallback()]
        
# Run training, with or without data augmentation.
if not data_augmentation:
    print('Not using data augmentation.')
    print('Training...')
    history = model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_valid, y_valid),
              shuffle=True,
              callbacks=callbacks)
else:
    print('Using real-time data augmentation.')
    print('Training...')
    # This will do preprocessing and realtime data augmentation:
    datagen = ImageDataGenerator(
        # set input mean to 0 over the dataset
        featurewise_center=False,
        # set each sample mean to 0
        samplewise_center=False,
        # divide inputs by std of dataset
        featurewise_std_normalization=False,
        # divide each input by its std
        samplewise_std_normalization=False,
        # apply ZCA whitening
        zca_whitening=False,
        # randomly rotate images in the range (deg 0 to 180)
        rotation_range=0,
        # randomly shift images horizontally
        width_shift_range=0.1,
        # randomly shift images vertically
        height_shift_range=0.1,
        # randomly flip images
        horizontal_flip=True,
        # randomly flip images
        vertical_flip=False)

    # Compute quantities required for featurewise normalization
    # (std, mean, and principal components if ZCA whitening is applied).
    datagen.fit(x_train)

    # Fit the model on the batches generated by datagen.flow().
    steps_per_epoch = math.ceil(len(x_train) / batch_size)
    history = model.fit_generator(datagen.flow(x_train, y_train, batch_size=batch_size),
                        validation_data=(x_valid, y_valid),
                        epochs=epochs, verbose=1, workers=4,
                        steps_per_epoch=steps_per_epoch,
                        callbacks=callbacks)

# Score trained model.
# scores = model.evaluate(x_test, y_test, verbose=1)
# print('Test loss:', scores[0])
# print('Test accuracy:', scores[1])

# Save training log
print('Saving epoch training log...')
train_error = history.history['loss']
valid_accuracy = history.history['val_acc']
logfile = '{}/training_log.csv'.format(save_dir)
f = open(logfile, 'w')
f.write('current_epoch,total_epochs,train_loss,validation_accuracy\n')
for i in range(len(train_error)):
    f.write('{},{},{},{}\n'.format(i+1, epochs, train_error[i], valid_accuracy[i]))
f.close()

print('Saving batch training log...')
f = open('batch_training_log.csv', 'w')
f.write('current_iteration,total_iteration,learning_rate,train_loss,train_accuracy\n')
total_iteration = int(math.ceil(x_train.shape[0]/batch_size)*epochs)
for i in range(len(batch_logs['iteration'])):
    f.write('{},{},{},{},{}\n'.format(batch_logs['iteration'][i], 
                                   total_iteration, 
                                   batch_logs['lr'][i], 
                                   batch_logs['loss'][i], 
                                   batch_logs['acc'][i]))
f.close()

import csv
with open('batch_training_log.csv', 'w') as f:  
    w = csv.DictWriter(f, batch_logs.keys())
    w.writeheader()
    w.writerow(batch_logs)

# Save index for combination
print('Writing index file and predict files...')
indexfile = '{}/index.csv'.format(save_dir)
f = open(indexfile, 'w')
window_size = int(epochs/top_k)
top_x = []
for i in range(0, top_k):
    top_x.append(np.argmax(valid_accuracy[i*snapshot_window_size:(i+1)*snapshot_window_size]) + i*snapshot_window_size)
top_v = [valid_accuracy[i] for i in top_x]
for x,v in zip(top_x, top_v):
    name = 'cifar10_{}_model-{:04d}.h5'.format(model_type, x+1)
    weight = v
    f.write('{},{}\n'.format(name, weight))
    # predicting
    filepath = os.path.join(save_dir, name)
    model.load_weights(filepath)
    predicts = model.predict(x_test)
    # Save predicts
    predictfile = '{}/prediction_{:04d}.csv'.format(save_dir, x+1)
    f1 = open(predictfile,'w')
    header = '0,1,2,3,4,5,6,7,8,9\n'
    f1.write(header)
    np.savetxt(f1, predicts, delimiter=",")
    f1.close()
f.close()

# Delete unwanted model files
print('Deleting unwanted model files...')
no_top_x = range(len(valid_accuracy))
no_top_x = list(set(no_top_x) - set(top_x))
for no_top_x_index in no_top_x:
    remove_filename = '{}/cifar10_{}_model-{:04d}.h5'.format(save_dir, model_type, no_top_x_index+1)
    os.remove(remove_filename)

# Save targets
print('Saving target file...')
targetfile = '{}/target.csv'.format(save_dir)
f2 = open(targetfile,'w')
header = '0,1,2,3,4,5,6,7,8,9\n'
f2.write(header)
np.savetxt(f2, y_test, delimiter=",")
f2.close()

Loading data...
Resampling training and validating data sets...
x_train shape: (50000, 32, 32, 3)
50000 train samples
4500 test samples
y_train shape: (50000, 1)
4500 valid samples
Building model...
ResNet20v1
Preparing callbacks...
Using real-time data augmentation.
Training...
Epoch 1/5

 Optimizer iteration 0, batch 0

 Learning rate 0.001, Model learning rate 0.0010000000474974513
  1/391 [..............................] - ETA: 1:09:31 - loss: 5.1286 - acc: 0.0703
 Optimizer iteration 1, batch 1

 Learning rate 0.0009999993511109622, Model learning rate 0.0009999993490055203

 Optimizer iteration 2, batch 2

 Learning rate 0.0009999974044455327, Model learning rate 0.0009999973699450493
  3/391 [..............................] - ETA: 23:13 - loss: 4.2290 - acc: 0.0807  
 Optimizer iteration 3, batch 3

 Learning rate 0.0009999941600087644, Model learning rate 0.0009999941103160381

 Optimizer iteration 4, batch 4

 Learning rate 0.0009999896178090784, Model learning rate 0.00099998

 45/391 [==>...........................] - ETA: 1:43 - loss: 2.3157 - acc: 0.2694
 Optimizer iteration 45, batch 45

 Learning rate 0.0009986865748457456, Model learning rate 0.000998686533421278

 Optimizer iteration 46, batch 46

 Learning rate 0.000998627578805444, Model learning rate 0.0009986276272684336
 47/391 [==>...........................] - ETA: 1:39 - loss: 2.2950 - acc: 0.2753
 Optimizer iteration 47, batch 47

 Learning rate 0.0009985672885492634, Model learning rate 0.0009985673241317272
 48/391 [==>...........................] - ETA: 1:37 - loss: 2.2860 - acc: 0.2772
 Optimizer iteration 48, batch 48

 Learning rate 0.0009985057042336898, Model learning rate 0.0009985057404264808
 49/391 [==>...........................] - ETA: 1:35 - loss: 2.2795 - acc: 0.2803
 Optimizer iteration 49, batch 49

 Learning rate 0.000998442826018569, Model learning rate 0.0009984428761526942
 50/391 [==>...........................] - ETA: 1:33 - loss: 2.2721 - acc: 0.2819
 Optimizer iterat

 90/391 [=====>........................] - ETA: 56s - loss: 2.0758 - acc: 0.3299
 Optimizer iteration 90, batch 90

 Learning rate 0.0009947531997255255, Model learning rate 0.0009947532089427114

 Optimizer iteration 91, batch 91

 Learning rate 0.0009946361664814943, Model learning rate 0.0009946362115442753
 92/391 [======>.......................] - ETA: 55s - loss: 2.0684 - acc: 0.3316
 Optimizer iteration 92, batch 92

 Learning rate 0.0009945178493815181, Model learning rate 0.0009945178171619773
 93/391 [======>.......................] - ETA: 54s - loss: 2.0635 - acc: 0.3329
 Optimizer iteration 93, batch 93

 Learning rate 0.000994398248732696, Model learning rate 0.000994398258626461
 94/391 [======>.......................] - ETA: 54s - loss: 2.0604 - acc: 0.3338
 Optimizer iteration 94, batch 94

 Learning rate 0.000994277364845458, Model learning rate 0.0009942774195224047

 Optimizer iteration 95, batch 95

 Learning rate 0.0009941551980335653, Model learning rate 0.0009941

186/391 [=============>................] - ETA: 26s - loss: 1.8512 - acc: 0.3941
 Optimizer iteration 186, batch 186

 Learning rate 0.0009777185133998268, Model learning rate 0.0009777185041457415

 Optimizer iteration 187, batch 187

 Learning rate 0.0009774801031602629, Model learning rate 0.00097748008556664
188/391 [=============>................] - ETA: 26s - loss: 1.8476 - acc: 0.3954
 Optimizer iteration 188, batch 188

 Learning rate 0.00097724045359428, Model learning rate 0.00097724050283432

 Optimizer iteration 189, batch 189

 Learning rate 0.0009769995653239022, Model learning rate 0.0009769995231181383
190/391 [=============>................] - ETA: 25s - loss: 1.8450 - acc: 0.3960
 Optimizer iteration 190, batch 190

 Learning rate 0.0009767574389743681, Model learning rate 0.0009767574956640601

 Optimizer iteration 191, batch 191

 Learning rate 0.0009765140751741306, Model learning rate 0.00097651407122612
192/391 [=============>................] - ETA: 25s - loss: 

234/391 [================>.............] - ETA: 18s - loss: 1.7857 - acc: 0.4153
 Optimizer iteration 234, batch 234

 Learning rate 0.0009648882429441257, Model learning rate 0.0009648882551118731

 Optimizer iteration 235, batch 235

 Learning rate 0.0009645911019582466, Model learning rate 0.0009645911050029099
236/391 [=================>............] - ETA: 18s - loss: 1.7836 - acc: 0.4159
 Optimizer iteration 236, batch 236

 Learning rate 0.0009642927551000749, Model learning rate 0.0009642927325330675

 Optimizer iteration 237, batch 237

 Learning rate 0.0009639932031439866, Model learning rate 0.0009639931959100068
238/391 [=================>............] - ETA: 17s - loss: 1.7819 - acc: 0.4168
 Optimizer iteration 238, batch 238

 Learning rate 0.0009636924468674854, Model learning rate 0.0009636924369260669

 Optimizer iteration 239, batch 239

 Learning rate 0.0009633904870512015, Model learning rate 0.0009633905137889087
240/391 [=================>............] - ETA: 17s 

...

163/391 [===========>..................] - ETA: 16s - loss: 0.9479 - acc: 0.7213
 Optimizer iteration 1727, batch 163

 Learning rate 3.192301211703952e-05, Model learning rate 3.192301301169209e-05
164/391 [===========>..................] - ETA: 16s - loss: 0.9474 - acc: 0.7217
 Optimizer iteration 1728, batch 164

 Learning rate 3.164040085872755e-05, Model learning rate 3.164040026604198e-05
165/391 [===========>..................] - ETA: 16s - loss: 0.9476 - acc: 0.7217
 Optimizer iteration 1729, batch 165

 Learning rate 3.1359005254054274e-05, Model learning rate 3.135900624329224e-05

 Optimizer iteration 1730, batch 166

 Learning rate 3.107882603339785e-05, Model learning rate 3.1078827305464074e-05
167/391 [===========>..................] - ETA: 15s - loss: 0.9468 - acc: 0.7221
 Optimizer iteration 1731, batch 167

 Learning rate 3.079986392397899e-05, Model learning rate 3.0799863452557474e-05
168/391 [===========>..................] - ETA: 15s - loss: 0.9464 - acc: 0.7226
 

253/391 [==================>...........] - ETA: 9s - loss: 0.9426 - acc: 0.7231 
 Optimizer iteration 1817, batch 253

 Learning rate 1.1434351466087178e-05, Model learning rate 1.1434351108619012e-05
254/391 [==================>...........] - ETA: 9s - loss: 0.9426 - acc: 0.7232
 Optimizer iteration 1818, batch 254

 Learning rate 1.1263698832695512e-05, Model learning rate 1.126369897974655e-05
255/391 [==================>...........] - ETA: 9s - loss: 0.9432 - acc: 0.7229
 Optimizer iteration 1819, batch 255

 Learning rate 1.1094314741816935e-05, Model learning rate 1.109431468648836e-05
256/391 [==================>...........] - ETA: 9s - loss: 0.9436 - acc: 0.7227
 Optimizer iteration 1820, batch 256

 Learning rate 1.0926199633097156e-05, Model learning rate 1.0926200047833845e-05
257/391 [==================>...........] - ETA: 9s - loss: 0.9431 - acc: 0.7230
 Optimizer iteration 1821, batch 257

 Learning rate 1.0759353942888573e-05, Model learning rate 1.0759354154288303e-05



300/391 [======================>.......] - ETA: 6s - loss: 0.9436 - acc: 0.7219
 Optimizer iteration 1864, batch 300

 Learning rate 4.791511883121713e-06, Model learning rate 4.791511855728459e-06

 Optimizer iteration 1865, batch 301

 Learning rate 4.680902408635335e-06, Model learning rate 4.680902293330291e-06
302/391 [======================>.......] - ETA: 6s - loss: 0.9432 - acc: 0.7220
 Optimizer iteration 1866, batch 302

 Learning rate 4.571578562679757e-06, Model learning rate 4.571578756440431e-06
303/391 [======================>.......] - ETA: 6s - loss: 0.9427 - acc: 0.7223
 Optimizer iteration 1867, batch 303

 Learning rate 4.463540629010998e-06, Model learning rate 4.4635407903115265e-06
304/391 [======================>.......] - ETA: 6s - loss: 0.9432 - acc: 0.7221
 Optimizer iteration 1868, batch 304

 Learning rate 4.356788888047747e-06, Model learning rate 4.356788849690929e-06
305/391 [======================>.......] - ETA: 6s - loss: 0.9426 - acc: 0.7224
 Optimiz

344/391 [=========================>....] - ETA: 3s - loss: 0.9420 - acc: 0.7222
 Optimizer iteration 1908, batch 344

 Learning rate 1.1442038431044854e-06, Model learning rate 1.1442037930464721e-06
345/391 [=========================>....] - ETA: 3s - loss: 0.9422 - acc: 0.7223
 Optimizer iteration 1909, batch 345

 Learning rate 1.0903861639985914e-06, Model learning rate 1.090386149371625e-06
346/391 [=========================>....] - ETA: 3s - loss: 0.9425 - acc: 0.7222
 Optimizer iteration 1910, batch 346

 Learning rate 1.0378634328099267e-06, Model learning rate 1.037863398778427e-06
347/391 [=========================>....] - ETA: 3s - loss: 0.9424 - acc: 0.7222
 Optimizer iteration 1911, batch 347

 Learning rate 9.866357858642206e-07, Model learning rate 9.866357686405536e-07
348/391 [=========================>....] - ETA: 3s - loss: 0.9422 - acc: 0.7223
 Optimizer iteration 1912, batch 348

 Learning rate 9.367033561257233e-07, Model learning rate 9.367033726448426e-07
349/39

389/391 [============================>.] - ETA: 0s - loss: 0.9439 - acc: 0.7214
 Optimizer iteration 1953, batch 389

 Learning rate 0.0009999941600087644, Model learning rate 0.0009999941103160381

 Optimizer iteration 1954, batch 390

 Learning rate 0.0009999896178090784, Model learning rate 0.0009999895701184869
391/391 [==============================] - 29s 74ms/step - loss: 0.9442 - acc: 0.7213 - val_loss: 1.0999 - val_acc: 0.6607

Epoch 00005: saving model to /home/ubuntu/Projects/hybrid-ensemble/model/run_200/cifar10_ResNet20v1_model-0005.h5
Saving epoch training log...
Saving batch training log...
Writing index file and predict files...
Deleting unwanted model files...
Saving target file...


In [9]:
logfile = '../model/run_100/training_log.csv'
df = pd.read_csv(logfile, header=0)
valid_accuracy = df['validation_accuracy'].values.tolist()
top_x = []
for i in range(0, 10):
    top_x.append(np.argmax(valid_accuracy[i*snapshot_window_size:(i+1)*snapshot_window_size]) + i*snapshot_window_size)


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=initial_lr),
              metrics=['accuracy'])
test_scores = []
for k in top_x:
    saved_model = '../model/run_100/cifar10_ResNet20v1_model-{:04d}.h5'.format(k+1)
    model.load_weights(saved_model)
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    test_scores.append(scores[1])
test_scores_1 = test_scores


4500/4500 [==============================] - 2s 337us/step
Test loss: 0.6096683431466421
Test accuracy: 0.8351111112170749
4500/4500 [==============================] - 1s 142us/step
Test loss: 0.5120277782943514
Test accuracy: 0.8691111110051473
4500/4500 [==============================] - 1s 142us/step
Test loss: 0.48847858993212384
Test accuracy: 0.8855555554495917
4500/4500 [==============================] - 1s 148us/step
Test loss: 0.4810516203906801
Test accuracy: 0.8931111111111111
4500/4500 [==============================] - 1s 148us/step
Test loss: 0.48305441366301644
Test accuracy: 0.8942222222222223
4500/4500 [==============================] - 1s 143us/step
Test loss: 0.48185164595974816
Test accuracy: 0.9008888888888889
4500/4500 [==============================] - 1s 141us/step
Test loss: 0.47659654211997987
Test accuracy: 0.9022222222222223
4500/4500 [==============================] - 1s 142us/step
Test loss: 0.49599758842256336
Test accuracy: 0.8997777777777778
4500/4500 [

In [10]:
print('Building model...')
if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)


model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=initial_lr),
              metrics=['accuracy'])

logfile = '../model/run_200/training_log.csv'
df = pd.read_csv(logfile, header=0)
valid_accuracy = df['validation_accuracy'].values.tolist()
version = 1
snapshot_window_size = 20
top_x = []
for i in range(0, 10):
    top_x.append(np.argmax(valid_accuracy[i*snapshot_window_size:(i+1)*snapshot_window_size]) + i*snapshot_window_size)


if version == 2:
    model = resnet_v2(input_shape=input_shape, depth=depth)
else:
    model = resnet_v1(input_shape=input_shape, depth=depth)

model.compile(loss='categorical_crossentropy',
              optimizer=Adam(lr=initial_lr),
              metrics=['accuracy'])
test_scores = []
for k in top_x:
    saved_model = '../model/run_200/cifar10_ResNet20v1_model-{:04d}.h5'.format(k+1)
    model.load_weights(saved_model)
    scores = model.evaluate(x_test, y_test, verbose=1)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])
    test_scores.append(scores[1])
test_scores_2 = test_scores

Building model...
4500/4500 [==============================] - 2s 411us/step
Test loss: 0.623283628516727
Test accuracy: 0.8286666666136847
4500/4500 [==============================] - 1s 151us/step
Test loss: 0.5248715903229184
Test accuracy: 0.864
4500/4500 [==============================] - 1s 149us/step
Test loss: 0.4969917231135898
Test accuracy: 0.8771111111111111
4500/4500 [==============================] - 1s 152us/step
Test loss: 0.4798820976946089
Test accuracy: 0.8871111111111111
4500/4500 [==============================] - 1s 149us/step
Test loss: 0.49182780093616907
Test accuracy: 0.8908888888888888
4500/4500 [==============================] - 1s 163us/step
Test loss: 0.4949813829925325
Test accuracy: 0.8993333333333333
4500/4500 [==============================] - 1s 151us/step
Test loss: 0.500278210149871
Test accuracy: 0.9004444444444445
4500/4500 [==============================] - 1s 148us/step
Test loss: 0.5235393349462085
Test accuracy: 0.8928888888888888
4500/4500 [=

In [4]:
import common_functions as cf
logfile = '../model/run_200/training_log.csv'
df = pd.read_csv(logfile, header=0)
valid_accuracy = df['validation_accuracy'].values.tolist()
version = 1
snapshot_window_size = 20
top_x = []
for i in range(1, 10):
    top_x.append(np.argmax(valid_accuracy[i*snapshot_window_size:(i+1)*snapshot_window_size]) + i*snapshot_window_size)

groundfile = '../model/run_200/target.csv' 
df_g = pd.read_csv(groundfile,header=0)
test_scores = []
for k in top_x:
    predfile = '../model/run_200/prediction_{:04d}.csv'.format(k+1)  
    df_p = pd.read_csv(predfile,header=0)     
    # compute confusion matrix
    cm = cf.confusion_matrix(df_g, df_p)
    tp, total = 0, 0
    for i in range(len(cm)):
        tp += cm[i,i]
        total += np.sum(cm[i])
    ea = tp/total*100
    print('Test accuracy:', ea)
    test_scores.append(ea)
test_scores_3 = test_scores

/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/numpy/core/fromnumeric.py:51: FutureWarning: 
The current behaviour of 'Series.argmax' is deprecated, use 'idxmax'
instead.
The behavior of 'argmax' will be corrected to return the positional
maximum in the future. For now, use 'series.values.argmax' or
'np.argmax(np.array(values))' to get the position of the maximum
row.
  return getattr(obj, method)(*args, **kwds)


Test accuracy: 86.4
Test accuracy: 87.71111111111111
Test accuracy: 88.71111111111111
Test accuracy: 89.08888888888889
Test accuracy: 89.93333333333334
Test accuracy: 90.04444444444445
Test accuracy: 89.28888888888889
Test accuracy: 90.42222222222223
Test accuracy: 90.02222222222223
